Use hydrolake depth as pre-computed depth for air2water model to constrain the parameter range

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import geopandas as gpd

In [2]:
hydrolakes = gpd.read_file("/mnt/Data_2tb/HydroSHEDS/HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10.shp")

In [3]:
cci_lakes = pd.read_csv("/home/xinchenh/Work/satbias_model/satlswt/data/ESA_CCI_static_lake_mask_v2_1km_UoR_metadata_fv2.1_06Oct2021_4laketemp.csv",
           index_col=0)

In [4]:
cci_lakes_gdf = gpd.GeoDataFrame(cci_lakes, geometry=gpd.points_from_xy(cci_lakes['LON CENTRE'], cci_lakes['LAT CENTRE']),
                                crs="epsg:4326")
# cci_lakes_gdf = gpd.read_file("/mnt/Data_2tb/laketemp_bias/lakes_cci_v2.1.0_shp/shapefile/lakescci_v2.1.0_data-availability.shp").set_index("id")

In [5]:
cci_lakes_gdf.crs = "epsg:4326"

In [6]:
cci_lakes_gdf.columns

Index(['NAME', 'COUNTRY', 'LAT CENTRE', 'LON CENTRE',
       'MAX DISTANCE TO LAND (KM)', 'LAT MIN BOX', 'LAT MAX BOX',
       'LON MIN BOX', 'LON MAX BOX', 'ID in GLOBOLAKES 1000 MASK',
       'ID in CGLOPS MASK', 'geometry'],
      dtype='object')

Remove lakes which are not located in ERA5-Land lake pixels

In [9]:
df_lake_temp = pd.read_csv("/mnt/Data_2tb/laketemp_bias/era5land/air_temp.csv", index_col=0)
lake_ids = [int(col) for col in df_lake_temp.columns]

In [10]:
# merge with hydrolake id
# select the largest lake area
cci_hydrolakes = cci_lakes_gdf.sjoin_nearest(hydrolakes, how = "left")
cci_hydrolakes = cci_hydrolakes.loc[lake_ids, ["NAME", "Hylak_id", "Depth_avg", "Lake_area"]]
cci_hydrolakes

/home/xinchenh/anaconda3/envs/zonal/lib/python3.11/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,NAME,Hylak_id,Depth_avg,Lake_area
CCI ID,,,,
799,Hawizeh marshes,175570,0.6,2.55
3114,loch Ness,13036,61.9,51.56
7889,lough Melvin,13375,7.7,20.84
2516,loch Lomond,13169,34.0,68.45
12262,loch Leven,13140,4.1,13.26
...,...,...,...,...
473,Shishmaref inlet and Ikpek lagoon and Cowpac...,403015,1.7,0.11
309,Zzzz,783,0.3,776.39
141,Baird inlet and Kaghasuk lake,622927,1.9,0.95


In [11]:
cci_hydrolakes.to_csv("data/cci_lakes_hydrolake_depth.csv")